In [1]:
## Imports
import pandas as pd
import requests


In [2]:
print('Jupyter is working!')

Jupyter is working!


In [5]:
material_data = {}
method = 'TRACI'

material_data.update({'Carrot':10})
print(material_data)
material_data.update({method:{'mini':'dict'}})
print(material_data)
material_data['data'] = "this"
print(material_data.keys())

{'Carrot': 10}
{'Carrot': 10, 'TRACI': {'mini': 'dict'}}
dict_keys(['Carrot', 'TRACI', 'data'])


In [ ]:
#created to not scroll down

In [ ]:
[
  {
    "material_id": "CON-001",
    "validity": {
      "gte": "2023-01-01T00:00:00",
      "lte": "2024-12-31T23:59:59"
    },
    "display_name": "High-Strength Concrete",
    "source": "EPD Database X",
    "declared_unit": "m3",
    "background_database": "Ecoinvent 3.8",
    "description": "High-strength concrete for structural applications",
    "A4_transportation": {
      "mode": 1.0,
      "distance": {
        "qty": 100.0,
        "unit": "km"
      }
    },
    "service_life": 50,
    "waste_rate": 0.05,
    "density": {
      "qty": 2400.0,
      "unit": "kg/m3"
    },
    "impacts": {
      "traci_21": {
        "gwp_total": {
          "A1_A3": 300.5,
          "C2": 10.2,
          "C3": 5.1,
          "C4": 2.3,
          "D1": -15.7
        }
      }
    },
    "characteristics": {}
  },
  {
    "material_id": "STL-001",
    "validity": {
      "gte": "2023-06-01T00:00:00",
      "lte": "2025-05-31T23:59:59"
    },
    "display_name": "Structural Steel",
    "source": "Industry Average EPD",
    "declared_unit": "kg",
    "A4_transportation": {
      "mode": 2.0,
      "distance": {
        "qty": 200.0,
        "unit": "km"
      }
    },
    "service_life": 75,
    "waste_rate": 0.02,
    "density": {
      "qty": 7850.0,
      "unit": "kg/m3"
    },
    "item_mass": {
      "qty": 1.0,
      "unit": "kg"
    },
    "impacts": {
      "EN15804": {
        "gwp_total": {
          "A1_A3": 2.1,
          "C2": 0.1,
          "C3": 0.05,
          "C4": 0.01,
          "D1": -0.5
        },
        "gwp_fossil": {
          "A1_A3": 2.0,
          "C2": 0.1,
          "C3": 0.04,
          "C4": 0.01,
          "D1": -0.48
        }
      }
    },
    "characteristics": {}
  },
  {
    "material_id": "INS-001",
    "validity": {
      "gte": "2023-03-15T00:00:00",
      "lte": "2024-03-14T23:59:59"
    },
    "display_name": "Glass Wool Insulation",
    "source": "Manufacturer EPD",
    "declared_unit": "m2",
    "background_database": "GaBi 2021",
    "description": "Glass wool insulation for thermal and acoustic purposes",
    "service_life": 40,
    "waste_rate": 0.1,
    "thickness": {
      "qty": 100.0,
      "unit": "mm"
    },
    "density": {
      "qty": 32.0,
      "unit": "kg/m3"
    },
    "impacts": {
      "traci_21": {
        "gwp_total": {
          "A1_A3": 1.8,
          "C2": 0.05,
          "C3": 0.02,
          "C4": 0.01,
          "D1": -0.3
        }
      },
      "EN15804": {
        "gwp_total": {
          "A1_A3": 1.9,
          "C2": 0.06,
          "C3": 0.025,
          "C4": 0.015,
          "D1": -0.35
        }
      }
    },
    "characteristics": {}
  }
]

In [ ]:
from datetime import datetime
import requests
#category = ['GWP-total','GWP-biogenic','GWP-fossil','GWP-IOBC']

headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+auth_key
}

product_epd_url = "https://buildingtransparency.org/api/epds"
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"

EPDs = []
EC3_epds = []
product_epds = []
none = "Unmapped"
page = 0

for x in range(0,10):
    if x % 250 == 0:
        page =+ 1
        EC3_epds.extend(get_page(page))


    date_validity_ends_str = EC3_epds[x]['date_validity_ends']
    date_validity_ends = datetime.strptime(date_validity_ends_str, '%Y-%m-%d')

    if date_validity_ends < datetime.now():
        print(f"EPD {x} expired, skipping...")
        continue

    xpdID = EC3_epds[x]['open_xpd_uuid']
    response_impact = requests.get(impact_info_url+xpdID, headers=headers)
    impact = response_impact.json()

    material_data = {} 

    ############################### DATA PATHS #################################
    material_data['material_id'] = EC3_epds[x]['category']['id']
    material_data.update({'validity':{'gte':EC3_epds[x]['category']['created_on'], 'lte':date_validity_ends_str}})





    
    EPDname = EC3_epds[x]['category']['display_name']
    desc = EC3_epds[x]['category']['description']
    seLi = EC3_epds[x]['reference_service_life']
    pcr_name = EC3_epds[x]['pcr']['name']
    pcr_id = EC3_epds[x]['pcr']['id']
    short_link = EC3_epds[x]['pcr']['short_link']
    created_on = EC3_epds[x]['category']['created_on']

    thckqty = EC3_epds[x]['thickness']
    thckU = EC3_epds[x]['thickness_per_declared_unit']

    tMode = EC3_epds[x]['category']['default_transport_mode']
    dist = EC3_epds[x]['category']['default_distance']

    dens = EC3_epds[x]['density']
    sdens = EC3_epds[x]['specific_density']

    unit = EC3_epds[x]['category']['declared_unit']
    lcia = list(EC3_epds[x]['pcr']['lcia_requirements'].keys())[0]
    



    material_data = {
    "material_id": EPDID,
    "validity": {
      "gte": created_on,
      "lte": date_validity_ends_str
    },
    "display_name": EPDname,
    "source": "Product Specific EPDs",
    "declared_unit": unit,
    "A4_transportation": {
      "mode": tMode,
      "distance": {
        "qty": dist,
        "unit": "km"
      }
    },
    "service_life": seLi,
    "waste_rate": none,
    "density": {
      "qty": dens,
      "unit": "kg/m3"
    },
    "item_mass": {
      "qty": 1.0,
      "unit": "kg"
    },
    "impacts": {
      "EN15804": {
        "gwp_total": {
          "A1_A3": 2.1,
          "C2": 0.1,
          "C3": 0.05,
          "C4": 0.01,
          "D1": -0.5
        },
        "gwp_fossil": {
          "A1_A3": 2.0,
          "C2": 0.1,
          "C3": 0.04,
          "C4": 0.01,
          "D1": -0.48
        }
      }
    },
    "characteristics": {}
  }
    
    EPDs.append(material_data)
    print(f'json #{x} is done')

